In [1]:
pip install python-dotenv pandas

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached pandas-2.3.1-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
Using cached pandas-2.3.1-cp310-cp310-win_amd64.whl (11.3 MB)
Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/5 [pytz]
   ---------------------------------------- 0/5 [pytz]
   ---------------------------------------- 0/5 [pytz]
   ---------------------------------------- 0/5 [pytz]
   ---------------------------------------- 0/5 [pytz]
   -------- ------------------------------- 1/5 [tzdata]
   -------- -------------------

In [11]:
import os, json, re, uuid, glob
from pathlib import Path
from typing import List, Dict

import pandas as pd
from dotenv import load_dotenv

# ---- 1. Load Environment Variables ----
# Point directly to your .env file (absolute path)
env_path = Path(r"C:\Users\kalva\AI_Projects\HomeShield_AI") / ".env"
load_dotenv(env_path)

# ---- 2. Required Variables Check ----
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY", 
    "PINECONE_API_KEY"
]

missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    raise RuntimeError(f"Missing env vars: {missing}")

# ---- 3. Path Configuration ----
POLICY_DIR = Path(r"C:\Users\kalva\AI_Projects\HomeShield_AI\policies_docs")
CUSTOMERS_CSV = Path(r"C:\Users\kalva\AI_Projects\HomeShield_AI\homeshield_sample_data\customers.csv")
EVAL_PAIRS = Path(r"C:\Users\kalva\AI_Projects\HomeShield_AI\homeshield_sample_data\evaluation_pairs.jsonl")

# ---- 4. Validation ----
print("Azure endpoint configured:", bool(os.getenv("AZURE_OPENAI_ENDPOINT")))
print("Policy directory exists:", POLICY_DIR.exists())
print("Customer data exists:", CUSTOMERS_CSV.exists())

Azure endpoint configured: True
Policy directory exists: True
Customer data exists: True


In [3]:
pip install openai pinecone

  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp310-cp310-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached pinecone_plugin_assistant-1.7.0-py3-none-any.whl.metadata (28 kB)

In [12]:
import os
from openai import AzureOpenAI
from pinecone import Pinecone

# ----- Secure Configuration -----
def get_env_var(name, default=None):
    """Safely get environment variable with validation"""
    value = os.environ.get(name, default)
    if value is None and default is None:
        raise ValueError(f"Missing required environment variable: {name}")
    return value

# Azure OpenAI Config
AZURE_OPENAI_ENDPOINT = get_env_var("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = get_env_var("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VER = get_env_var("AZURE_OPENAI_API_VERSION")
EMBED_MODEL = get_env_var("AZURE_OPENAI_EMBEDDING_DEPLOYMENT") 
CHAT_MODEL = get_env_var("AZURE_OPENAI_CHAT_DEPLOYMENT")

# Pinecone Config
PC_API_KEY = get_env_var("PINECONE_API_KEY")
PINECONE_INDEX = get_env_var("PINECONE_INDEX")
PINECONE_REGION = get_env_var("PINECONE_REGION")

# ----- Client Initialization -----
# Azure OpenAI Client
oai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VER,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

# Pinecone Client
pc = Pinecone(api_key=PC_API_KEY)

# ----- Index Verification -----
print(f"Connecting to existing Pinecone index: {PINECONE_INDEX}")
index = pc.Index(PINECONE_INDEX)

# Verify index configuration matches your expectations
index_stats = index.describe_index_stats()
print("\nIndex Configuration:")
print(f"- Dimensions: {index_stats.dimension}")
print(f"- Metric: {index_stats.metric}")
print(f"- Vector Count: {index_stats.total_vector_count}")
print(f"- Embedding Model: {EMBED_MODEL}")

# Safety check for embedding dimension
expected_dim = 1536  # text-embedding-3-small uses 1536 dimensions
if index_stats.dimension != expected_dim:
    print(f"\nWARNING: Index dimension ({index_stats.dimension}) doesn't match expected ({expected_dim})")
    print("You may need to recreate your index with the correct dimensions")
else:
    print("\nIndex validation successful - ready for operations!")

Connecting to existing Pinecone index: homeshield-policies

Index Configuration:
- Dimensions: 1536
- Metric: cosine
- Vector Count: 0
- Embedding Model: text-embedding-3-small

Index validation successful - ready for operations!


In [4]:
pip install langchain-openai langchain-pinecone

  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached langchain_tests-0.3.20-py3-none-any.whl.metadata (3.3 kB)
  Using cached pytest-8.4.1-py3-none-any.whl.metadata (7.7 kB)
  Using cached pytest_asyncio-0.26.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached syrupy-4.9.1-py3-none-any.whl.metadata (38 kB)
  Using cached pytest_socket-0.7.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached pytest_benchmark-5.1.0-py3-none-any.whl.metadata (25 kB)
  Using cached pytest_codspeed-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached pytest_recording-0.13.4-py3-none-any.whl.metadata (11 kB)
  Using cached vcrpy-7.0.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached aiohttp_retry-2.9.1-py3-none-any.whl.metadata (8.8 kB)
  Using cached iniconfig-2.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached pluggy-1.6.0-py3-non

In [13]:
# Block 3 — LangChain + Azure embeddings + Pinecone VectorStore

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import os

# Azure LLM + Embeddings (deployment names come from your .env)
llm = AzureChatOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT"],
    temperature=0,
)

embeddings = AzureOpenAIEmbeddings(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"],
)

# Ensure Pinecone index exists with dim=1536 (Azure text-embedding-3-small)
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
INDEX_NAME   = os.environ["PINECONE_INDEX"]
REGION       = os.environ["PINECONE_REGION"]
DIM          = 1536  # Azure text-embedding-3-small

if INDEX_NAME not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=INDEX_NAME,
        dimension=DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=REGION),
    )

# LangChain VectorStore handle (keep namespace stable)
NAMESPACE = "policies"
vectorstore = PineconeVectorStore(index_name=INDEX_NAME, embedding=embeddings, namespace=NAMESPACE)

print("LangChain ready → index:", INDEX_NAME, "| namespace:", NAMESPACE)


LangChain ready → index: homeshield-policies | namespace: policies


In [9]:
# Run this in a Jupyter notebook cell
%pip install langchain-community langchain-core langchain-text-splitters pinecone-client

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached pinecone_client-6.0.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.3.27-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.1-py3-none-any.whl (8.1 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 9.7 MB/s eta 0:00:00
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
Using cached pydantic_settings-2.10.1-py3-

In [29]:
# Adapted “simple strategy” for our project (TXT + Azure + metadata + namespace)
# with namespace wipe + 'text' metadata for LangChain compatibility

import os, time
from pathlib import Path
from dotenv import load_dotenv
from tqdm import tqdm

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from pinecone import Pinecone

# --- Load env if needed (keeps your existing .env setup) ---
# load_dotenv(Path(r"C:\Users\kalva\AI_Projects\HomeShield_AI") / ".env")

# --- 1) Load TXT policies ---
raw_docs = []
for p in POLICY_DIR.glob("*.txt"):
    raw_docs.extend(TextLoader(str(p), encoding="utf-8").load())
print("Loaded TXT files:", len(raw_docs))

# --- 2) Chunk ---
splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=120, add_start_index=True)
chunks = splitter.split_documents(raw_docs)
print("Chunks:", len(chunks))

# --- 3) Attach metadata from filename: LHG_<Plan>_<STATE>_<YEAR>.txt ---
def parse_meta_from_filename(path_str: str):
    stem = Path(path_str).stem
    parts = stem.split("_")
    if len(parts) >= 4 and parts[0] == "LHG":
        _, plan, state, year = parts[:4]
        try:
            return {"plan": plan, "state": state, "effective_year": int(year)}
        except:
            return {}
    return {}

for i, d in enumerate(chunks):
    src = d.metadata.get("source", "")
    d.metadata.update(parse_meta_from_filename(src))
    d.metadata.setdefault("source", Path(src).name or "unknown.txt")
    d.metadata.setdefault("page", (i // 5) + 1)
    d.metadata.setdefault("section", "policy")
    d.metadata.setdefault("chunk_id", f"{d.metadata['source']}-{i:04d}")

# --- 4) Embeddings (Azure, 1536-dim) ---
emb = AzureOpenAIEmbeddings(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"],
)

# --- 5) Pinecone client & index handle ---
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index(INDEX_NAME)


# --- 6) Upsert with throttling (avoid 429s) & include 'text' in metadata ---
BATCH = 64
BASE_SLEEP = 3.0
MAX_RETRIES = 6

def embed_texts(texts):
    # LangChain embeddings returns list of vectors
    return emb.embed_documents(texts)

def upsert_batch(batch, start_id):
    texts = [d.page_content for d in batch]
    vecs = embed_texts(texts)
    vectors = []
    for j, d in enumerate(batch):
        vectors.append({
            "id": f"hs-{start_id + j:08d}",
            "values": vecs[j],
            "metadata": {
                **d.metadata,
                "text": d.page_content,   # <-- critical: store text so LangChain can rebuild Documents
            }
        })
    index.upsert(vectors=vectors, namespace=NAMESPACE)

def add_batch_with_backoff(batch, start_id):
    delay = BASE_SLEEP
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            upsert_batch(batch, start_id)
            return
        except Exception as e:
            msg = str(e)
            is_429 = ("429" in msg) or ("Too Many Requests" in msg) or ("rate" in msg.lower())
            if not is_429 or attempt == MAX_RETRIES:
                print(f"[ERROR] Upsert failed (attempt {attempt}): {e}")
                raise
            # exponential backoff with cap
            sleep_for = min(delay * (1.8 ** (attempt - 1)), 30.0)
            print(f"[WARN] 429 rate limit; retrying in {sleep_for:.1f}s (attempt {attempt}/{MAX_RETRIES})")
            time.sleep(sleep_for)

total = len(chunks)
for i in tqdm(range(0, total, BATCH), desc="Upserting"):
    batch = chunks[i:i + BATCH]
    add_batch_with_backoff(batch, start_id=i)
    time.sleep(BASE_SLEEP)  # steady pacing
    if (i // BATCH) % 10 == 0:
        print(f"Progress: {min(i + BATCH, total)}/{total}")

print(f"✅ Completed upsert to Pinecone index='{INDEX_NAME}', namespace='{NAMESPACE}'.")


Loaded TXT files: 300
Chunks: 4500


Upserting:   1%|▏         | 1/71 [00:05<06:02,  5.18s/it]

Progress: 64/4500


INFO:httpx:HTTP Request: POST https://homeshield.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
Upserting:  15%|█▌        | 11/71 [00:49<04:26,  4.44s/it]

Progress: 704/4500


...

Progress: 3264/4500


INFO:httpx:HTTP Request: POST https://homeshield.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
Upserting:  86%|████████▌ | 61/71 [04:31<00:44,  4.42s/it]

Progress: 3904/4500


INFO:httpx:HTTP Request: POST https://homeshield.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
Upserting: 100%|██████████| 71/71 [05:15<00:00,  4.44s/it]

Progress: 4500/4500
✅ Completed upsert to Pinecone index='homeshield-policies', namespace='policies'.


In [17]:
# Bind LangChain to the same Pinecone index/namespace you just upserted into
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(
    index_name=INDEX_NAME,
    embedding=emb,           # <-- reuse your AzureOpenAIEmbeddings instance
    namespace=NAMESPACE      # "policies"
)
print("VectorStore ready:", INDEX_NAME, "| ns:", NAMESPACE)


VectorStore ready: homeshield-policies | ns: policies


In [25]:
import sys, importlib.metadata as md

def v(pkg):
    try:
        return md.version(pkg)
    except md.PackageNotFoundError:
        return "not installed"

print("python:", sys.version.split()[0])
print("langchain:", v("langchain"))
print("langchain-openai:", v("langchain-openai"))
print("langchain-community:", v("langchain-community"))
print("langchain-pinecone:", v("langchain-pinecone"))
print("pinecone-client:", v("pinecone-client"))
print("openai:", v("openai"))


python: 3.10.18
langchain: 0.3.27
langchain-openai: 0.3.30
langchain-community: 0.3.27
langchain-pinecone: 0.2.11
pinecone-client: 6.0.0
openai: 1.99.9


In [30]:
# 100% safe MMR using VectorStore API directly (no compressors involved)

def retrieve_mmr(question: str, plan: str, state: str, year: int, k: int = 8, fetch_k: int = 24, lambda_mult: float = 0.5):
    """
    Uses vectorstore.max_marginal_relevance_search to get diverse chunks.
    Applies strict metadata filter so we only search the correct policy set.
    """
    return vectorstore.max_marginal_relevance_search(
        question,
        k=k,
        fetch_k=fetch_k,
        lambda_mult=lambda_mult,
        filter={"plan": plan, "state": state, "effective_year": year},
    )

def format_context(docs):
    lines = []
    for i, d in enumerate(docs):
        src  = d.metadata.get("source", "")
        page = d.metadata.get("page", "")
        lines.append(f"[{i}] {src} p.{page}\n{d.page_content}")
    return "\n\n---\n\n".join(lines)

# smoke test
_test = retrieve_mmr("HVAC compressor coverage", "Gold", "PA", 2025)
print("MMR hits:", len(_test))
print(_test[0].metadata, _test[0].page_content[:160])


INFO:httpx:HTTP Request: POST https://homeshield.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


MMR hits: 8
{'chunk_id': 'C:\\Users\\kalva\\AI_Projects\\HomeShield_AI\\policies_docs\\LHG_Gold_PA_2025.txt-1126', 'effective_year': 2025.0, 'page': 226.0, 'plan': 'Gold', 'section': 'policy', 'source': 'C:\\Users\\kalva\\AI_Projects\\HomeShield_AI\\policies_docs\\LHG_Gold_PA_2025.txt', 'start_index': 478.0, 'state': 'PA'} SECTION – HVAC
- Compressor is covered for mechanical and electrical failures under the Gold plan, subject to exclusions listed below. Pre-existing conditions a


In [35]:
docs = vectorstore.max_marginal_relevance_search(
    "HVAC compressor coverage",
    k=8, fetch_k=50, lambda_mult=0.5,
    filter={"plan": "Gold", "state": "PA", "effective_year": 2025},
)
print("hits:", len(docs))
if docs:
    print(docs[0].metadata, "\n", docs[0].page_content[:500])

INFO:httpx:HTTP Request: POST https://homeshield.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


hits: 8
{'chunk_id': 'C:\\Users\\kalva\\AI_Projects\\HomeShield_AI\\policies_docs\\LHG_Gold_PA_2025.txt-1126', 'effective_year': 2025.0, 'page': 226.0, 'plan': 'Gold', 'section': 'policy', 'source': 'C:\\Users\\kalva\\AI_Projects\\HomeShield_AI\\policies_docs\\LHG_Gold_PA_2025.txt', 'start_index': 478.0, 'state': 'PA'} 
 SECTION – HVAC
- Compressor is covered for mechanical and electrical failures under the Gold plan, subject to exclusions listed below. Pre-existing conditions and improper installation are excluded. Wear items (e.g., filters, belts) are excluded unless otherwise stated.
- Evaporator Coil is covered for mechanical and electrical failures under the Gold plan, subject to exclusions listed below. Pre-existing conditions and improper installation are excluded. Wear items (e.g., filters, belts) are ex
